## Inpatient activity

### Mitigation data

We have the table `nhp.raw_data.apc_mitigators` which we will use for collecting data on mitigation. However, this goes to the granularity of 92 mitigators, whilst we are concerning ourselves with three *types* of mitigation, as well as the counterfactual: re-direction and substitution ("reSu"), prevention ("prev") and de-adoption ("deAdopt"). There are also instances in which a specific hospital admission is in scope of more than mitigator; in this case we choose assign the admission according to a hierachy: prevention > de-adoption > re-direction and substitution; else "none". Finally, the mitigators have rates which we must sample from to reflect the fact that now all cases meeting the [definitions for each mitigator](https://connect.strategyunitwm.nhs.uk/nhp/project_information/modelling_methodology/activity_mitigators/inpatient_activity_mitigators.html) will actually be relevant.

The below query creates a view `activity_mitigation_table` where the sampling, grouping and hierachy have been effected.

In [0]:
%sql
select * from su_data.reference.icd10_codes

In [0]:
%sql
CREATE VIEW activity_mitigation_table AS
WITH sampled_data AS (
    SELECT
        epikey,
        sample_rate,
        type,
        strategy,
        CASE
            WHEN strategy IN (
                'alcohol_partially_attributable_acute', 'alcohol_partially_attributable_chronic', 'alcohol_wholly_attributable', 'falls_related_admissions',
                'obesity_related_admissions', 'smoking') THEN 'prev'
            WHEN strategy IN (
                'evidence_based_interventions_ent', 'evidence_based_interventions_general_surgery', 'evidence_based_interventions_gi_surgical',
                'evidence_based_interventions_msk', 'evidence_based_interventions_urology', 'evidence_based_interventions_vascular_varicose_veins') THEN 'deAdopt'
            WHEN strategy IN (
                'ambulatory_care_conditions_acute', 'ambulatory_care_conditions_chronic', 'ambulatory_care_conditions_vaccine_preventable',
                'eol_care_2_days', 'eol_care_3_to_14_days', 'frail_elderly_high', 'frail_elderly_intermediate', 'intentional_self_harm',
                'medically_unexplained_related_admissions', 'medicines_related_admissions_explicit', 'medicines_related_admissions_implicit_anti-diabetics',
                'medicines_related_admissions_implicit_benzodiasepines', 'medicines_related_admissions_implicit_diurectics',
                'medicines_related_admissions_implicit_nsaids', 'raid_ae', 'readmission_within_28_days', 'zero_los_no_procedure_adult',
                'zero_los_no_procedure_child') THEN 'reSu'
            ELSE 'none'
        END AS mitigation_type
    FROM nhp.raw_data.apc_mitigators
    WHERE sample_rate >= RAND()
),
ranked_data AS (
    SELECT
        *,
        ROW_NUMBER() OVER (
            PARTITION BY epikey
            ORDER BY
                CASE mitigation_type
                    WHEN 'prev' THEN 1
                    WHEN 'deAdopt' THEN 2
                    WHEN 'reSu' THEN 3
                    ELSE 4
                END
        ) AS rn
    FROM sampled_data
)
SELECT * FROM ranked_data WHERE rn = 1;


### Note on missing mitigators
In the query above we 30 specific strategies that have been allocated to one of the three types of mitigation. 

However, there are 33 inpatient activity avoidance mitigators (derived below), meaning there are three missing:

- _cancelled_operations_
- _virtual_wards_activity_avoidance_ari_
- _virtual_wards_activity_avoidance_heart_failure_


In [0]:
%sql
select distinct strategy from nhp.raw_data.apc_mitigators where type = "activity_avoidance"

### ICD10 chapters

One of the things we want the data by is ICD10 chapter of the principal diagnosis.

There is an existing reference table `Su_data.reference.icd10_codes` but we just need to create a new copy `icd10_codes_gabriel` as the main refence table does not have the chapters correctedly tabulated.

In [0]:
%sql
CREATE OR REPLACE VIEW icd10_codes_gabriel as
SELECT
    *,
    CASE 
        WHEN chapter_description = 'Certain infectious and parasitic diseases' THEN 'I'
        WHEN chapter_description = 'Neoplasms' THEN 'II'
        WHEN chapter_description = 'Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism' THEN 'III'
        WHEN chapter_description = 'Endocrine, nutritional and metabolic diseases' THEN 'IV'
        WHEN chapter_description = 'Mental and behavioural disorders' THEN 'V'
        WHEN chapter_description = 'Diseases of the nervous system' THEN 'VI'
        WHEN chapter_description = 'Diseases of the eye and adnexa' THEN 'VII'
        WHEN chapter_description = 'Diseases of the ear and mastoid process' THEN 'VIII'
        WHEN chapter_description = 'Diseases of the circulatory system' THEN 'IX'
        WHEN chapter_description = 'Diseases of the respiratory system' THEN 'X'
        WHEN chapter_description = 'Diseases of the digestive system' THEN 'XI'
        WHEN chapter_description = 'Diseases of the skin and subcutaneous tissue' THEN 'XII'
        WHEN chapter_description = 'Diseases of the musculoskeletal system and connective tissue' THEN 'XIII'
        WHEN chapter_description = 'Diseases of the genitourinary system' THEN 'XIV'
        WHEN chapter_description = 'Pregnancy, childbirth and the puerperium' THEN 'XV'
        WHEN chapter_description = 'Certain conditions originating in the perinatal period' THEN 'XVI'
        WHEN chapter_description = 'Congenital malformations, deformations and chromosomal abnormalities' THEN 'XVII'
        WHEN chapter_description = 'Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified' THEN 'XVIII'
        WHEN chapter_description = 'Injury, poisoning and certain other consequences of external causes' THEN 'XIX'
        WHEN chapter_description = 'External causes of morbidity and mortality' THEN 'XX'
        WHEN chapter_description = 'Factors influencing health status and contact with health services' THEN 'XXI'
        WHEN chapter_description = 'Codes for special purposes' THEN 'XXII'
    END AS chapter_number
FROM strategyunit.reference.icd10_codes 

In [0]:
%sql
select distinct chapter_code from 

### Extracting data

The aim is to extract counts of activity (spells) by the following groups:

-   financial year
-   admission type / pod
-   age
-   sex
-   region
-   ICD-10 chapter
-   deprivation quintile
-   mitigation type

In [0]:
%sql
describe activity_mitigation_table

In [0]:
%sql
select * from activity_mitigation_table

In [0]:
%sql
SELECT
  a.fyear,
  a.group AS pod,
  a.age,
  a.resgor_ons,
  CASE
    WHEN a.sex = '1' THEN 'm'
    ELSE 'f'
  END AS sex,
  d.chapter_number,
  a.imd_quintile,
  b.mitigation_type,
  COUNT(a.epikey) AS activity  
 FROM nhp.raw_data.apc a
 LEFT JOIN 
  activity_mitigation_table b
  ON a.epikey = b.epikey
LEFT JOIN
  icd10_codes_gabriel d
  ON a.primary_diagnosis = d.icd10
WHERE
  a.fyear BETWEEN '201112' AND '201920'
GROUP BY
  a.fyear,
  a.group,
  a.age,
  a.resgor_ons,
  CASE
    WHEN a.sex = '1' THEN 'm'
    ELSE 'f'
  END,
  d.chapter_number,
  a.imd_quintile,
  b.mitigation_type;

In [0]:
%sql
-- Aim here is to just the counts of mitigation each year and just confirm that elective de-adoption is indeed rising the fastest
SELECT
  a.fyear,
  a.group AS pod,
  CASE
    WHEN isnull(b.mitigation_type) THEN 'none'
    ELSE b.mitigation_type
  END AS mitigation_type,
  COUNT(a.epikey) AS activity  
FROM nhp.raw_data.apc a 
LEFT JOIN 
  activity_mitigation_table b
  ON a.epikey = b.epikey
GROUP BY
  a.fyear,
  a.group,
  CASE
    WHEN isnull(b.mitigation_type) THEN 'none'
    ELSE b.mitigation_type
  END


## Outpatient activity

## Outpatients

Consulting the lookup there are 12 outpatients. Each of these will be 4 variants, so in reality it's 3 OPA groupings.

These are the ones in Stevem's lookup:
- consultant_to_consultant_reduction_adult_non-surgical
- consultant_to_consultant_reduction_adult_surgical
- consultant_to_consultant_reduction_child_non-surgical
- consultant_to_consultant_reduction_child_surgical
- followup_reduction_adult_non-surgical
- followup_reduction_adult_surgical
- followup_reduction_child_non-surgical
- followup_reduction_child_surgical


We're off by one set of four:
- gp_referred_first_attendance_reduction_adult_non-surgical
- gp_referred_first_attendance_reduction_adult_surgical
- gp_referred_first_attendance_reduction_child_non-surgical
- gp_referred_first_attendance_reduction_child_surgical

### Creating the mitigators table

### Extraction

All the OPA mitigators are categorised as de-adoption, so there is no need to create this mitigator type variable in the table; we will simply check if the episode appears in that table and if so categorise it as de-adoption.

NOTE: in this case we do use a join to take advantage of the new table that Tom has created via workflow: `nhp.raw_data.opa_mitigators`.

We do need to deal with attendances that appear more than once so we count distinct on `attendkey`.

This is still breaking down so we split the data into two extracts: one for 2011-12 to 2014-15 and another for 2015-16 to 2019-20. This isn't an arbritrary division as 2015-16 is the point at which the region variable is properly populated.

In [0]:
%sql
SELECT 
  'opa' AS pod,
  a.fyear, 
  a.age, 
  a.sex,
  a.type,
  a.imd_quintile,
  a.resgor_ons,
  CASE
    WHEN isnotnull(b.strategy) THEN 'deAdopt'
    ELSE 'none'
  END AS mitigation_type,
  COUNT(distinct a.attendkey) AS activity
FROM
  nhp.raw_data.opa a
LEFT JOIN
  nhp.raw_data.opa_mitigators b
  ON a.attendkey = b.attendkey AND a.provider = b.provider AND a.fyear = b.fyear
WHERE
  a.fyear between 201112 and 201415
GROUP BY
  a.fyear, 
  a.age, 
  a.sex,
  a.type,
  a.imd_quintile,
  a.resgor_ons,
  CASE
    WHEN isnotnull(b.strategy) THEN 'deAdopt'
    ELSE 'none'
  END;

In [0]:
%sql
SELECT 
  'opa' AS pod,
  a.fyear, 
  a.age, 
  a.sex,
  a.type,
  a.imd_quintile,
  a.resgor_ons,
  CASE
    WHEN isnotnull(b.strategy) THEN 'deAdopt'
    ELSE 'none'
  END AS mitigation_type,
  COUNT(distinct a.attendkey) AS activity
FROM
  nhp.raw_data.opa a
LEFT JOIN
  nhp.raw_data.opa_mitigators b
  ON a.attendkey = b.attendkey AND a.provider = b.provider AND a.fyear = b.fyear
WHERE
  a.fyear between 201516 and 201819
GROUP BY
  a.fyear, 
  a.age, 
  a.sex,
  a.type,
  a.imd_quintile,
  a.resgor_ons,
  CASE
    WHEN isnotnull(b.strategy) THEN 'deAdopt'
    ELSE 'none'
  END;

In [0]:
%sql
SELECT 
  'opa' AS pod,
  a.fyear, 
  a.age, 
  a.sex,
  a.type,
  a.imd_quintile,
  a.resgor_ons,
  CASE
    WHEN isnotnull(b.strategy) THEN 'deAdopt'
    ELSE 'none'
  END AS mitigation_type,
  COUNT(distinct a.attendkey) AS activity
FROM
  nhp.raw_data.opa a
LEFT JOIN
  nhp.raw_data.opa_mitigators b
  ON a.attendkey = b.attendkey AND a.provider = b.provider AND a.fyear = b.fyear
WHERE
  a.fyear = 201920
GROUP BY
  a.fyear, 
  a.age, 
  a.sex,
  a.type,
  a.imd_quintile,
  a.resgor_ons,
  CASE
    WHEN isnotnull(b.strategy) THEN 'deAdopt'
    ELSE 'none'
  END;

## A&E activity

### Categorising the mitigators for ED 

The aea mitigators are all redirection and substitution, so we can quite simply deal with that in the script.

We do need to deal with arrivals with appear more than once in the mitigators data. This can be done by counting distinct on `aekey`.

In [0]:
%sql
describe nhp.raw_data.ecds

In [0]:
%sql

CREATE VIEW aea_mitigators_data AS

SELECT
  ecds.key,
  concat('frequent_attenders_', ecds.type) strategy
FROM
  nhp.raw_data.ecds ecds
WHERE
  ecds.is_frequent_attender = 1

UNION ALL

SELECT
  ecds.key,
  concat('left_before_seen_', ecds.type) strategy
FROM
  nhp.raw_data.ecds ecds
WHERE
  ecds.is_left_before_treatment = 1

UNION ALL

SELECT
  ecds.key,
  concat('low_cost_discharged_', ecds.type) strategy
FROM
  nhp.raw_data.ecds ecds
WHERE
  ecds.is_low_cost_referred_or_discharged = 1

UNION ALL

SELECT
  ecds.key,
  concat('discharged_no_treatment_', ecds.type) strategy
FROM
  nhp.raw_data.ecds ecds
WHERE
  ecds.is_discharged_no_treatment = 1

### Extracting the ED data

In [0]:
%sql
SELECT 
  'aae' AS pod, 
  a.fyear, 
  a.age, 
  a.sex, 
  a.group,
  a.imd_quintile,
  a.resgor_ons, 
  CASE
    WHEN EXISTS(SELECT 1 FROM aea_mitigators_data WHERE key = a.key) THEN 'reSu'
    ELSE 'none'
  END AS mitigation_type,
  COUNT(DISTINCT a.key) AS activity
FROM
  nhp.raw_data.ecds a
WHERE
  fyear BETWEEN 201112 AND 201920
  AND aedepttype = '01'
GROUP BY
  a.fyear, 
  a.age, 
  a.sex, 
  a.group,
  a.imd_quintile,
  a.resgor_ons,
  CASE
    WHEN EXISTS(SELECT 1 FROM aea_mitigators_data WHERE key = a.key) THEN 'reSu'
    ELSE 'none'
  END;

#### New SQL query for region in 201920 

In [0]:
%sql
-- Let local_authority_to_region_lookup be a table that we have loaded in. 
-- We might need to substitute this at some point depending on how loading imported tables works.
SELECT 
  'aae' AS pod, 
  a.fyear, 
  a.age, 
  a.sex, 
  a.group,
  a.imd_quintile,
  CASE
    WHEN a.fyear < 201920 THEN a.resgor_ons
    ELSE b.RGN19CD
  END AS resgor_ons,
  CASE
    WHEN EXISTS(SELECT 1 FROM aea_mitigators_data WHERE key = a.key) THEN 'reSu'
    ELSE 'none'
  END AS mitigation_type,
  COUNT(DISTINCT a.key) AS activity
FROM
  nhp.raw_data.ecds a
LEFT JOIN
  strategyunit.default.lad_19_rgn_19_en_lu_948748_b_0_eaa_54_fe_888_a_604_b_126_f_5_e_672_4818463568216355194 b
  ON a.resladst_ons = b.LAD19CD
WHERE
  fyear BETWEEN 201112 AND 201920
  AND aedepttype = '01'
GROUP BY
  a.fyear, 
  a.age, 
  a.sex, 
  a.group,
  a.imd_quintile,
  CASE
    WHEN a.fyear < 201920 THEN a.resgor_ons
    ELSE b.RGN19CD
  END,
  CASE
    WHEN EXISTS(SELECT 1 FROM aea_mitigators_data WHERE key = a.key) THEN 'reSu'
    ELSE 'none'
  END;

## Table showing strategies in base year and final year by pod

In [0]:
%sql 
SELECT
  a.fyear,
  CASE 
    WHEN a.group = "elective" THEN "IpElec"
    WHEN a.group = "maternity" THEN "IpMat"
    ELSE "IpEmer"
  END AS pod,
  CASE 
    WHEN b.mitigation_type IS NULL THEN 'none'
    ELSE b.mitigation_type
  END AS mitigation_type,
  CASE 
    WHEN b.mitigation_type IS NULL OR b.mitigation_type = "none" THEN "none"
    ELSE b.strategy
  END AS strategy,
  COUNT(a.epikey) AS activity  
 FROM nhp.raw_data.apc a
 LEFT JOIN 
  activity_mitigation_table b
  ON a.epikey = b.epikey
WHERE a.fyear in (201112,201920)
GROUP BY
  a.fyear,
  CASE 
    WHEN a.group = "elective" THEN "IpElec"
    WHEN a.group = "maternity" THEN "IpMat"
    ELSE "IpEmer"
  END, 
  CASE 
    WHEN b.mitigation_type IS NULL THEN 'none'
    ELSE b.mitigation_type
  END,
  CASE 
    WHEN b.mitigation_type IS NULL OR b.mitigation_type = "none" THEN "none"
    ELSE b.strategy
  END
UNION ALL
SELECT 
  a.fyear,
  'opAtt' as pod,
  CASE 
    WHEN b.strategy IS NULL THEN 'none'
    ELSE  'deAdopt'
  END AS mitigation_type,
  CASE 
    WHEN b.strategy IS NULL THEN 'none'
    ELSE  b.strategy
  END AS strategy,
  count(distinct(a.attendkey)) AS activity 
FROM nhp.raw_data.opa a
LEFT JOIN nhp.raw_data.opa_mitigators b
  ON a.attendkey = b.attendkey AND a.provider = b.provider AND a.fyear = b.fyear
WHERE
  a.fyear in (201112, 201920)
GROUP BY
  a.fyear,
  CASE 
    WHEN b.strategy IS NULL THEN 'none'
    ELSE  'deAdopt'
  END,
  CASE 
    WHEN b.strategy IS NULL THEN 'none'
    ELSE  b.strategy
  END
UNION ALL
SELECT 
  a.fyear,
  'edAtt' as pod,
  CASE 
    WHEN b.strategy IS NULL THEN 'none'
    ELSE  'reSu'
  END AS mitigation_type,
  CASE 
    WHEN b.strategy IS NULL THEN 'none'
    ELSE  b.strategy
  END AS strategy,
  count(distinct(a.key)) AS activity
FROM nhp.raw_data.ecds a
LEFT JOIN aea_mitigators_data b
  ON a.key = b.key
WHERE
  a.fyear in (201112, 201920) AND a.aedepttype = '01'
GROUP BY
  a.fyear,
  CASE 
    WHEN b.strategy IS NULL THEN 'none'
    ELSE  'reSu'
  END,
  CASE 
    WHEN b.strategy IS NULL THEN 'none'
    ELSE  b.strategy
  END;

  

### Issue with inpatient mitigation counts not matching
The totals for each pod match the rest of the figures but they are very slightly different in how the numbers are apportioned. This is probably to do with the way that NULLs are dealt with.

However, when looking at the discrepancies three chunks below, there are only three inpatient avoidance mitigators where there is a discrepancy but these are the ones we specifically did not cover and as such shouldn't matter:

virtual_wards_activity_avoidance_heart_failure
cancelled_operations
virtual_wards_activity_avoidance_ari


In [0]:
%sql
-- Aim here is to just the counts of mitigation each year and just confirm that elective de-adoption is indeed rising the fastest
SELECT
  'converting_nulls'  as sql_script,
  a.group AS pod,
  CASE
    WHEN b.mitigation_type IS NULL THEN 'none'
    ELSE b.mitigation_type
  END AS mitigation_type,
CASE 
    WHEN b.mitigation_type IS NULL OR b.mitigation_type = "none" THEN "none"
    ELSE b.strategy
  END AS strategy,
  COUNT(a.epikey) AS activity  
FROM nhp.raw_data.apc a 
LEFT JOIN 
  activity_mitigation_table b
  ON a.epikey = b.epikey
WHERE a.fyear = 201112
GROUP BY
  a.group,
  CASE
    WHEN isnull(b.mitigation_type) THEN 'none'
    ELSE b.mitigation_type
  END,
CASE 
    WHEN b.mitigation_type IS NULL OR b.mitigation_type = "none" THEN "none"
    ELSE b.strategy
  END
UNION ALL
SELECT
  'no changes' as sql_script,
  a.group AS pod,
  b.mitigation_type,
  b.strategy,
  COUNT(a.epikey) AS activity  
 FROM nhp.raw_data.apc a
 LEFT JOIN 
  activity_mitigation_table b
  ON a.epikey = b.epikey
WHERE a.fyear = 201112
GROUP BY
  a.group,
  b.mitigation_type,
  b.strategy;


In [0]:
%sql
CREATE TEMPORARY VIEW inpatient_mitig_QA as
select 
  *,
  CASE
    WHEN mitigation_type is null THEN 'none'
    ELSE mitigation_type
  END AS mitigation_type_1,
CASE 
    WHEN mitigation_type IS NULL OR mitigation_type = "none" THEN "none"
    ELSE strategy
  END AS strategy_1
 from activity_mitigation_table

In [0]:
%sql
SELECT distinct type, strategy, mitigation_type, mitigation_type_1, strategy_1 FROM inpatient_mitig_QA WHERE strategy <> strategy_1;

### Inpatient - without preserving the hierachy of mitigators

For this one we actually want to show the counts without the hierachy imposed. This means that the counts for each strategy within a mitigation class (reSu, deAdopt, prev) will NOT sum to the total for the mitigation class, given that admissions might fall in scope of more than one mitigator. 

In [0]:
%sql
CREATE VIEW sampled_data AS
  SELECT
      epikey,
      sample_rate,
      type,
      strategy,
      CASE
          WHEN strategy IN (
              'alcohol_partially_attributable_acute', 'alcohol_partially_attributable_chronic', 'alcohol_wholly_attributable','falls_related_admissions',
              'obesity_related_admissions', 'smoking') THEN 'prev'
          WHEN strategy IN (
              'evidence_based_interventions_ent', 'evidence_based_interventions_general_surgery','evidence_based_interventions_gi_surgical',
              'evidence_based_interventions_msk', 'evidence_based_interventions_urology','evidence_based_interventions_vascular_varicose_veins') THEN 'deAdopt'
          WHEN strategy IN (
              'ambulatory_care_conditions_acute', 'ambulatory_care_conditions_chronic','ambulatory_care_conditions_vaccine_preventable',
              'eol_care_2_days', 'eol_care_3_to_14_days', 'frail_elderly_high', 'frail_elderly_intermediate','intentional_self_harm',
              'medically_unexplained_related_admissions', 'medicines_related_admissions_explicit','medicines_related_admissions_implicit_anti-diabetics',
              'medicines_related_admissions_implicit_benzodiasepines', 'medicines_related_admissions_implicit_diurectics',
              'medicines_related_admissions_implicit_nsaids', 'raid_ae', 'readmission_within_28_days','zero_los_no_procedure_adult',
              'zero_los_no_procedure_child') THEN 'reSu'
          ELSE 'none'
      END AS mitigation_type
  FROM nhp.raw_data.apc_mitigators
  WHERE sample_rate >= RAND()

In [0]:
%sql
select * from sampled_data;

In [0]:
%sql
SELECT
  a.fyear,
  CASE 
    WHEN a.group = "elective" THEN "IpElec"
    WHEN a.group = "maternity" THEN "IpMat"
    ELSE "IpEmer"
  END AS pod,
  CASE 
    WHEN b.mitigation_type IS NULL THEN 'none'
    ELSE b.mitigation_type
  END AS mitigation_type,
  CASE 
    WHEN b.mitigation_type IS NULL OR b.mitigation_type = "none" THEN "none"
    ELSE b.strategy
  END AS strategy,
  COUNT(a.epikey) AS activity  
 FROM nhp.raw_data.apc a
 LEFT JOIN 
  sampled_data b
  ON a.epikey = b.epikey
WHERE a.fyear in (201112,201920)
GROUP BY
  a.fyear,
  CASE 
    WHEN a.group = "elective" THEN "IpElec"
    WHEN a.group = "maternity" THEN "IpMat"
    ELSE "IpEmer"
  END, 
  CASE 
    WHEN b.mitigation_type IS NULL THEN 'none'
    ELSE b.mitigation_type
  END,
  CASE 
    WHEN b.mitigation_type IS NULL OR b.mitigation_type = "none" THEN "none"
    ELSE b.strategy
  END 
UNION ALL
SELECT 
  a.fyear, 
  CASE 
    WHEN a.group = "elective" THEN "IpElec"
    WHEN a.group = "maternity" THEN "IpMat"
    ELSE "IpEmer"
  END,
  'TOTAL' AS mitigation_type,
  'TOTAL' AS strategy,
  COUNT(a.epikey) AS activity
FROM nhp.raw_data.apc a
LEFT JOIN sampled_data b
ON a.epikey = b.epikey
WHERE a.fyear in (201112,201920)
GROUP BY
  a.fyear,
  CASE 
    WHEN a.group = "elective" THEN "IpElec"
    WHEN a.group = "maternity" THEN "IpMat"
    ELSE "IpEmer"
  END;


### Issue with double counting of opa and aae mitigators

The counts of non-mitig are identical to the rest of the analysis. However, the mitig data is much higher which is because of there being attendances which appear in more than one strategy.

The below query groups the aae mitig data so that the four variants adult/child ambulance/walk-in are amalgamated.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW aea_mitig_strategies_grouped AS 
SELECT 
  *, 
  REGEXP_REPLACE(strategy, '_(child|adult)_(walk-in|ambulance)$', '') AS stem_strategy 
FROM 
  aea_mitigators_data
WHERE 
  strategy LIKE '%_walk-in' OR strategy LIKE '%_ambulance';

-- Then:
--SELECT * 
--FROM aea_mitig_strategies_grouped
--WHERE (key, stem_strategy) IN (
--  SELECT key, stem_strategy
--  FROM aea_mitig_strategies_grouped
--  GROUP BY key, stem_strategy
--  HAVING COUNT(*) > 1
--)
--ORDER BY key;

select * from aea_mitig_strategies_grouped order by key

## Previous SQL queries on Strategy Unit Server

We have the following three scripts for categorising strategies from the previous analysis



### in-patient prevention

```sql
use StrategicWorking

--drop table dbo.sw_ip_mitigatable_prevention


select EPIKEY, fyear

into dbo.sw_ip_mitigatable_prevention

from [HESData].[nhp_strategies].[ip_alcohol_partially_attributable] 
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_alcohol_wholly_attributable]
where fraction >= RAND(CHECKSUM(NewId()))


union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_falls_related_admissions]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_obesity_related_admissions]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_smoking]
where fraction >= RAND(CHECKSUM(NewId()))

```

### in-patient re-direction and substitution

```sql
use StrategicWorking

--drop table dbo.sw_ip_mitigatable_redirect_substitute


select EPIKEY, fyear

into dbo.sw_ip_mitigatable_redirect_substitute

from [HESData].[nhp_strategies].[ip_ambulatory_care_conditions_acute]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_ambulatory_care_conditions_chronic]
where fraction >= RAND(CHECKSUM(NewId()))


union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_ambulatory_care_conditions_vaccine_preventable]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_eol_care]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_frail_elderly]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_intentional_self_harm]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_medically_unexplained_related_admissions]
where fraction >= RAND(CHECKSUM(NewId()))


union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_medicines_related_admissions_explicit]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_medicines_related_admissions_implicit_anti_diabetics]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_medicines_related_admissions_implicit_benzodiasepines]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_medicines_related_admissions_implicit_diurectics]
where fraction >= RAND(CHECKSUM(NewId()))


union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_medicines_related_admissions_implicit_nsaids]
where fraction >= RAND(CHECKSUM(NewId()))


union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_raid_ae]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_readmission_within_28_days]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_zero_los_no_procedure]
where fraction >= RAND(CHECKSUM(NewId()))
```



### in-patient de-adoption

```sql
use StrategicWorking

--drop table dbo.sw_ip_mitigatable_rationing


select EPIKEY, fyear

into dbo.sw_ip_mitigatable_rationing

from [HESData].[nhp_strategies].[ip_evidence_based_interventions_ent]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_evidence_based_interventions_general_surgery]
where fraction >= RAND(CHECKSUM(NewId()))


union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_evidence_based_interventions_gi_surgical]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_evidence_based_interventions_msk]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_evidence_based_interventions_urology]
where fraction >= RAND(CHECKSUM(NewId()))

union


select EPIKEY, fyear
from [HESData].[nhp_strategies].[ip_evidence_based_interventions_vasuclar_varicose_veins]
where fraction >= RAND(CHECKSUM(NewId()))
```



### outpatient de-adoption
```sql
use StrategicWorking

--drop table dbo.sw_op_mitigatable_rationing

select attendkey, fyear

into dbo.sw_op_mitigatable_rationing

from [HESData].[nhp_strategies].[op_followup_reduction]
where fraction >= RAND(CHECKSUM(NewId()))

union


select attendkey, fyear
from [HESData].[nhp_strategies].[op_consultant_to_consultant_referrals]
where fraction >= RAND(CHECKSUM(NewId()))
```

### emergency department re-direction and substitution
```sql
use StrategicWorking

--drop table dbo.sw_ed_mitigatable_redirect_substitute


select aekey, fyear

into dbo.sw_ed_mitigatable_redirect_substitute

from [HESData].[nhp_strategies].[aae_frequent_attender]
where fraction >= RAND(CHECKSUM(NewId()))

union


select aekey, fyear
from [HESData].[nhp_strategies].[aae_left_before_treatment]
where fraction >= RAND(CHECKSUM(NewId()))


union


select aekey, fyear
from [HESData].[nhp_strategies].[aae_low_cost_discharged]
where fraction >= RAND(CHECKSUM(NewId()))
```

In [0]:
%sql

CREATE VIEW su_data.nhp.aae_ungrouped_mitigators AS

SELECT
  aae.aekey,
  concat('frequent_attenders_', aae.type) strategy
FROM
  su_data.nhp.aae_ungrouped aae
WHERE
  aae.is_frequent_attender = 1

UNION ALL

SELECT
  aae.aekey,
  concat('left_before_seen_', aae.type) strategy
FROM
  su_data.nhp.aae_ungrouped aae
WHERE
  aae.is_left_before_treatment = 1

UNION ALL

SELECT
  aae.aekey,
  concat('low_cost_discharged_', aae.type) strategy
FROM
  su_data.nhp.aae_ungrouped aae
WHERE
  aae.is_low_cost_referred_or_discharged = 1

UNION ALL

SELECT
  aae.aekey,
  concat('discharged_no_treatment_', aae.type) strategy
FROM
  su_data.nhp.aae_ungrouped aae
WHERE
  aae.is_discharged_no_treatment = 1